# 향상된 역할 특화 및 단계별 추론 에이전트 시스템 (Gemma3-4B 적용)

이 노트북은 Google Colab TPU 환경에서 Gemma3-4B Instruction-tuned 모델을 사용한 역할 특화 에이전트 시스템을 구축합니다.

## 변경/개선
- 모델 ID를 2B에서 3-4B IT 버전으로 변경
- TPU 환경에서는 Transformers 대신 JAX/Flax 기반 로딩 예시 추가
- GPU 환경에서는 BitsAndBytesConfig를 Q4_0으로 설정
- Accelerate를 활용한 TPU 장치 매핑 예시 추가
- 역할 라우팅 로직 개선 제안

In [ ]:
# ## 1. 환경 설정 및 라이브러리 설치
!pip install -q -U transformers accelerate bitsandbytes torch torch_xla jax jaxlib flax

In [ ]:
# ## 2. TPU 사용 확인 및 공통 설정
import os, torch

on_tpu = False
if os.environ.get('COLAB_TPU_ADDR'):
    on_tpu = True
    print("✅ TPU 환경 감지됨: JAX/Flax 로딩 사용 권장 (bf16).")
else:
    print("🚫 TPU 미감지: GPU/CPU 환경으로 로딩.")

In [ ]:
# ## 3A. [TPU 전용] JAX/Flax 로딩 예시 (Gemma3-4B)
if on_tpu:
    os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
    os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

    from transformers import FlaxAutoModelForCausalLM, AutoTokenizer

    model_id = "google/gemma-3-4b-it"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = FlaxAutoModelForCausalLM.from_pretrained(
        model_id,
        dtype="bfloat16",
        _do_init=False
    )
    print(f"{model_id} Flax 로딩 완료 (dtype=bfloat16).")

    import jax
    @jax.pmap
    def infer_fn(input_ids, attention_mask):
        return model(input_ids=input_ids, attention_mask=attention_mask).logits

    print("✅ JAX/Flax pmap inference 준비 완료.")

In [ ]:
# ## 3B. [GPU/CPU 공통] Transformers + BitsAndBytes 로딩 예시
else:
    from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
    from accelerate import init_empty_weights, infer_auto_device_map

    model_id = "google/gemma-3-4b-it"
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    use_quant = torch.cuda.is_available()
    quant_cfg = None
    if use_quant:
        quant_cfg = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16
        )
        print("🔧 GPU 양자화 활성화: Q4_0 (nf4)")

    with init_empty_weights():
        dummy = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quant_cfg)

    device_map = infer_auto_device_map(dummy, no_split_module_classes=["GPTJBlock"])
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quant_cfg,
        device_map=device_map,
        torch_dtype=torch.bfloat16 if not use_quant else None
    )
    print(f"{model_id} Transformers 로딩 완료 on {device_map}.")

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        do_sample=True,
    )
    print("✅ Transformers pipeline 생성 완료.")

In [ ]:
# ## 4. 향상된 역할 특화 에이전트 정의
agent_roles = {
    # 기존과 동일하게 정의
}

In [ ]:
# ## 5. 요청 라우팅 로직 개선
import re
def route_request(user_query):
    q = user_query.lower()
    if re.search(r"\b(파이썬 코드|generate code)\b", q):
        return "code_generator"
    if re.search(r"\b(요약해줘|summarize)\b", q):
        return "summarizer"
    if re.search(r"\b(영어로 번역)\b", q):
        return "translator_ko_en"
    return "general_assistant"

In [ ]:
# ## 6. 추론 실행 함수
def generate_response_agentic(pipeline_instance, role_id, user_query):
    if on_tpu:
        inputs = tokenizer(user_query, return_tensors="jax", padding=True)
        logits = infer_fn(inputs["input_ids"], inputs["attention_mask"])
        return "⚠️ TPU 샘플링 함수는 직접 구현해야 합니다."
    else:
        return pipe(f"{agent_roles[role_id]['system_prompt']}\n{user_query}")[0]["generated_text"]

In [ ]:
# ## 7. 데모 실행
if not on_tpu:
    for q in ["간단한 파이썬 함수 생성해줘", "다음 텍스트를 요약해줘: ..."]:
        role = route_request(q)
        print(f"\n▶ {role} 처리:", generate_response_agentic(pipe, role, q))
else:
    print("▶ TPU 환경: JAX inference 코드 작성 후 테스트 필요.")

## 8. 결론 및 향후 과제

- Gemma3-4B 모델로 확장하여 더 풍부한 컨텍스트 및 성능 확보
- TPU 환경에서는 Transformers 대신 JAX/Flax 사용 권장 (bf16)
- GPU 환경에서는 BitsAndBytesConfig를 Q4_0(nf4)으로 설정해 메모리 절약
- Accelerate의 init_empty_weights + infer_auto_device_map으로 대형 모델 자동 분산 배치
- 향후: 
  - TPU용 샘플링/디코딩 로직 구현  
  - LangChain/AI agent 프레임워크와 결합한 실제 에이전트 제어  
  - NLU intent 분류 모델을 통한 정교한 라우팅